In [1]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
api_key = open("C://Users//Marcial//Desktop//desktop_openai.txt").read()
model = ChatOpenAI(openai_api_key=api_key)

In [9]:
# STEP ONE - Import Parser, 
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

In [10]:
# Step two - format instructions

In [11]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [12]:
reply = "red, blue, green"

In [15]:
output_parser.parse(reply)

['red', 'blue', 'green']

In [42]:
human_template = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [43]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [59]:
model_request = chat_prompt.format_prompt(request='write a poem about animals',
                          format_instructions=output_parser.get_format_instructions()).to_messages()

In [60]:
result = model(model_request)

In [63]:
# print(result.content)

In [65]:
# output_parser.parse(result.content)

In [66]:
from langchain.output_parsers import DatetimeOutputParser

In [67]:
output_parser = DatetimeOutputParser()

In [69]:
# output_parser.get_format_instructions()

In [76]:
template_text = "{request}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(template_text)


system_prompt = SystemMessagePromptTemplate.from_template("You always reply to questions only in datetime patterns.")

In [82]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [83]:
model_request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                          format_instructions=output_parser.get_format_instructions()).to_messages()

In [84]:
result = model(model_request,temperature=0)

In [85]:
result.content

'The 13th Amendment was ratified in the US on December 6, 1865.\n\nThe datetime string that matches the given pattern is: "1865-12-06T00:00:00.000000Z"'

In [97]:
from langchain.output_parsers import OutputFixingParser

In [98]:
misformatted = result.content

In [99]:
misformatted

'The 13th Amendment was ratified in the US on December 6, 1865.\n\nThe datetime string that matches the given pattern is: "1865-12-06T00:00:00.000000Z"'

In [100]:
output_parser 

In [114]:
new_parser = OutputFixingParser.from_llm(parser=output_parser,llm=model)
new_parser.parse(misformatted)

datetime.datetime(1865, 12, 6, 0, 0)

In [115]:
from langchain.output_parsers import PydanticOutputParser

In [116]:
from pydantic import BaseModel, Field

In [117]:
class Scientist(BaseModel):
    
    name: str = Field(description='Name of a Scientist')
    discoveries: list = Field(description="Python list of discoveries")

In [118]:
parser = PydanticOutputParser(pydantic_object=Scientist)

In [119]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "Name of a Scientist", "type": "string"}, "discoveries": {"title": "Discoveries", "description": "Python list of discoveries", "type": "array", "items": {}}}, "required": ["name", "discoveries"]}
```


In [120]:
human_prompt = HumanMessagePromptTemplate.from_template("{request}\n{format_instructions}")

In [121]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [122]:
request = chat_prompt.format_prompt(request="Tell me about a famous scientist",
                                    format_instructions=parser.get_format_instructions()).to_messages()

In [123]:
result = model(request,temperature=0)

In [126]:
type(parser.parse(result.content))

__main__.Scientist